$  % These are commands to simply writing vectors in bold. 
   \newcommand{\RR}{\mathbb{R}}
   \newcommand{\R}{\mathbb{R}}
   \newcommand{\B}[2][\varepsilon]{ B_{#1} \left( #2 \right) }
   \newcommand{\vec}[1]{ \boldsymbol{\mathbf{#1}} }
   \newcommand{\Z}{\mathbb{Z}}
   \newcommand{\C}{\mathbb{C}}
   \newcommand{\tr}[1]{\mathrm{tr}\left( #1 \right)}
   \newcommand{\ran}{\mathrm{ran}}
   \newcommand{\MC}[1]{\mathcal{M}_{#1}\left(\mathbb{C}\right)}
   \newcommand{\MR}[1]{\mathcal{M}_{#1}\left(\mathbb{R}\right)}
   \newcommand{\bkt}[1]{\left\langle #1 \right\rangle}
   \newcommand{\brc}[1]{\left\lbrace #1 \right\rbrace}
   \newcommand{\set}[2]{\left\lbrace #1 \middle\vert #2 \right\rbrace}
   \newcommand{\sfrac}[2]{\mathstrut^{#1}/_{#2}}
   \newcommand{\ra}{\rightarrow}
   \newcommand{\hip}[2]{\left\langle {#1},{#2} \right\rangle}
   \newcommand{\norm}[1]{\left\Vert {#1} \right\Vert}
   \newcommand{\hnorm}[1]{\left\Vert {#1} \right\Vert_2}
   \DeclareMathOperator*{\argmax}{argmax} % thin space, limits underneath in displays
   \DeclareMathOperator*{\argmin}{argmin} % no space, limits underneath in displays
$    

<h2 align = 'center'> Fall 2023: Operations Research </h2>
<h3 align = 'center'>  Noah Borquaye</h3>
<h4 align = 'center'>  Portfolio Optimization Problem</h4>
<p>&nbsp;</p>







First, we load some packages and do some configuration:

In [25]:
%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np 
%precision 9

from pandas import DataFrame
import warnings
warnings.simplefilter('ignore')

%run -i SimplexTools.py

The following commands are now available

* __add_method__ = adds a method to an instance of any class
* __var__ = creates a variable for creating Tableaus
* __param__ = creates a parameter for creating Tableaus
* __Vars__ = creates var objects from a list or string
* __Params__ = creates param objects from a list or structure
* __LinearExpression__ = class for expressions for linear programs
* __LinearRelation__ = class for equations and inequalities
* __GaussPivot__ = Gaussian Row Operations pivot for a pandas DataFrame
* __Tableau__ = Maps linear program as dictionary to DataFrame with GaussPivot

Also, the following helper methods are available:
* __BasicSolution__ = prints basic solution for a given tableau
* __OrderBasis__ = Arranges the simplex tableau into [I,M] form, where I is the identity matrix and M is what is left over
* __Simplex__ = Create simplex Tableau for a linear program, adding slack or surplus variables if necessary.  The solve method only implements maximization. 

NOTE: __GaussPivot__ can be applied directly to any DataFrame


<Figure size 640x480 with 0 Axes>

## __Scenerio__

__1.__ Obtain and solve the following as the _Primal_ program: 
> A funds manager receives \\$100,000 from a
new client for the purchase of stocks and is required to choose between blue
chip stocks (BLUE), tech stocks(TECH), and high risk/high yield stocks
(HRHY). The expected rate of return on BLUE is \\$1 per share, the rate of
return on TECH is \\$2 per share, and the rate of return on HRHY is \\$15 per
share. Brokerage house rules require thatBrokerage house rules require that<br/>
&nbsp;&nbsp;&nbsp;&nbsp;   a. The fund manager is to purchase no more than 2,500 shares <br/>
&nbsp;&nbsp;&nbsp;&nbsp;   b. The amount of BLUE stocks purchased exceeds the total of the other stocks purchased <br/>
&nbsp;&nbsp;&nbsp;&nbsp;   c.  The total of HRHY stocks purchased never exceeds 10 percent of the BLUE stocks purchased<br/>
If the cost of BLUE stocks is \\$100 per share, the cost of TECH stocks is
\\$60 per share, and the cost of HRHY stocks is \\$40 per share, then how many
shares of each should the fund manager purchase in order to maximize the
expected return on investment. 

Obtain the dual program, solve using the method of your choice, and explain what the dual program means (interpret the variables and constraints). Then find the shadow prices and explain what they mean. In particular, interpret each shadow price as the "price" for increasing the availability of a resource by one unit.

### Solution

__Decision Variables__:


Let $x_1$ be the number of BLUE stocks purchased.\
Let $x_2$ be the number of TECH stocks purchased.\
Let $x_3$ be the number of HRHY stocks purchased.

We aim to maximize the expected return on investment:\
__Maximize:__ $x_1 + 2x_2 + 15x_3$

__Subject to the constraints:__

1. The fund manager is to purchase no more than 2,500 shares:\
$x_1 + x_2 + x_3 \leq 2500$

2. The amount of BLUE stocks purchased exceeds the total of the other stocks purchased:\
$x_1 \geq x_2 + x_3$

3. The total of HRHY stocks purchased never exceeds 10 percent of the BLUE stocks purchased:\
$x_3 \leq 0.1x_1$

4. To maximize expected returns, we must also take into account the cost of the stocks. BLUE stocks is $\$100$ per share, the cost of TECH stocks is $\$60$ per share, and the cost of HRHY stocks is $\$40$ per share and funds manager receives $\$100,000$ from a new client for the purchase of stocks:\
$100x_1 + 60x_2 + 40x_3 \leq 100000$

Finally, since the number of shares cannot be negative, we need to consider the non-negativity constraints:\
$x_1 \geq 0$, $x_2 \geq  0$, $x_3 \geq 0$




__The Primal program is:__


$$\begin{array}{ll}
maximize & z=x_1 + 2x_2 + 15x_3 \\ 
subject \; to \; & x_1 + x_2 + x_3 &\leq 2500 \\ 
\; & 100x_1 + 60x_2 + 40x_3 &\leq 100000 \\ 
\; & -x_1+x_2 + x_3 &\leq 0  \\ 
\; & -0.1x_1+ x_3 & \leq 0 \\
& x_1\geq 0,x_2\geq 0, x_3\geq 0
\end{array}
$$


Solving using __glpk__

In [26]:
%%script glpsol -m /dev/stdin

# declare problem variables
var x_1 >= 0;  # we can define properties for
var x_2 >= 0 ;  # the variables as we define them
var x_3 >= 0;
# declare the objective 
maximize z: x_1 + 2*x_2 + 15*x_3;       ## DON'T FORGET THOSE SEMICOLONS!!!!! 

# subject to the constraints
s.t.    constraint1:    x_1 + x_2 +x_3  <=  2500;
        constraint2:    100*x_1 + 60*x_2 +40*x_3  <=  100000;
        constraint3:    -1*x_1+x_2 + x_3 <= 0; 
        constraint4:    -0.1*x_1+ x_3 <= 0; 

solve;

# display results
display x_1, x_2,x_3, z;

end;

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 -m /dev/stdin
Reading model section from /dev/stdin...
20 lines were read
Generating z...
Generating constraint1...
Generating constraint2...
Generating constraint3...
Generating constraint4...
Model has been successfully generated
GLPK Simplex Optimizer, v4.65
5 rows, 3 columns, 14 non-zeros
Preprocessing...
4 rows, 3 columns, 11 non-zeros
Scaling...
 A: min|aij| =  1.000e-01  max|aij| =  1.000e+02  ratio =  1.000e+03
GM: min|aij| =  4.472e-01  max|aij| =  2.236e+00  ratio =  5.000e+00
EQ: min|aij| =  2.000e-01  max|aij| =  1.000e+00  ratio =  5.000e+00
Constructing initial basis...
Size of triangular part is 4
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (3)
*     3: obj =   2.721518987e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (102371 bytes)
Display statement at line 18
x_1.val = 632.911392405063
x_2.val = 569.620253164557
x_3.val = 63.291139240506






\
The solution to the primal program is $z^* = 2721.518$ and $(x_1^*,x_2^*,x_3^*) = (632.91, 569.62,  63.29 )$

Thus to maximize the expected return on investment, the fund manager should purchase $632.91$units of BLUE stocks, $569.62$units of TECH stocks and $63.29$units of HRHY stocks.

__The Dual program is__ 

$$\begin{array}{ll}
minimize \; & z = 2500y_1 + 100000y_2 + 0y_3 + 0y_4 \\ 
subject \; to \; & y_1 + 100y_2 - y_3 - 0.1y_4 \geq 1 \\ 
\;  & y_1 + 60y_2 + y_3 \geq 2  \\ 
\;  & y_1 + 40y_2 + y_3 + y_4\geq 15  \\ 
\;  & y_1\geq 0,y_2\geq 0, y_3\geq 0, y_4\geq 0
\end{array}
$$



Solving using __glpk__



In [1]:
%%script glpsol -m /dev/stdin

# declare problem variables
set I := 1..4;
var y{I} >= 0 ; 

# declare the objective 
minimize z: 2500*y[1] + 100000*y[2] + 0*y[3] + 0*y[4];  

# subject to the constraints
s.t.    constraint1:  y[1] + 100*y[2] - y[3] - 0.1*y[4] >= 1;
        constraint2:  y[1] + 60*y[2] + y[3] >= 2;
        constraint3:  y[1] + 40*y[2] + y[3] + y[4] >= 15;

# solve
solve;

# display results
display y,z;

end;

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 -m /dev/stdin
Reading model section from /dev/stdin...
20 lines were read
Generating z...
Generating constraint1...
Generating constraint2...
Generating constraint3...
Model has been successfully generated
GLPK Simplex Optimizer, v4.65
4 rows, 4 columns, 13 non-zeros
Preprocessing...
3 rows, 4 columns, 11 non-zeros
Scaling...
 A: min|aij| =  1.000e-01  max|aij| =  1.000e+02  ratio =  1.000e+03
GM: min|aij| =  4.472e-01  max|aij| =  2.236e+00  ratio =  5.000e+00
EQ: min|aij| =  2.000e-01  max|aij| =  1.000e+00  ratio =  5.000e+00
Constructing initial basis...
Size of triangular part is 3
      0: obj =   0.000000000e+00 inf =   7.376e+00 (3)
      3: obj =   2.721518987e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (118423 bytes)
Display statement at line 18
y[4].val = 13.5443037974684
y[3].val = 0.367088607594937
y[2].val = 0.0272151898734177
y[1].val = 0
z.val







\
We interprete the variables in the dual program as follows:\
Let $y_1$ denotes the total number of shares the fund manager can purchase.\
$~ ~~~~y_2$ denotes the total fund received from the client.\
$~~~~~y_3$ and $y_4$ both represent the constriants for the relationship between different shares.

The dual program yields\
$ z^*  = 2721.52$, which is optimal from first part of the problem.\
$ y_1^* = 0, y_2^* =0.03 , y_3^* =0.37 , y_4^*= 13.54  $



The shadow prices that optimize investment returns are\
$y_1 = 0.$ This means when the total number of shares is increased by 1 unit from 2500, the total return on investment does not increase.\
$y_2 = 0.03$ This means an increase in the available fund from $\$100000$ by one unit will lead to $\$0.03$ (3 cents) 
              increase in return on the investment.\
$ y_3 = 0.37, y_4 = 13.54$ are with zero resources and therefore have no effect on the investment returns.

